In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import json
from decouple import config
from urllib import request
import csv 
import datetime
import smtplib
from pandas import json_normalize
from sqlalchemy import create_engine
import psycopg2
from email.message import EmailMessage
from datetime import datetime, timedelta
import json 
import pandas as pd
from collections import defaultdict
from collections import ChainMap

In [2]:
LW_username = config('lw_username')
LW_password = config('lw_password')

LW_payload = {'username': LW_username, 'password': LW_password}
loginurl = ('https://osteostrong.org/auth')
LW_totals = []

with requests.session() as s:
    s.post(loginurl, data=LW_payload)
    LW_url = f'https://osteostrong.org/user/users?LocationId=00053'
    LW_r = s.get(LW_url).text
    jsondata=json.loads(LW_r)    
s.close()
metrics=[]
# x=int(input())
current_date = datetime.now()
cd=current_date.strftime("%m-%d-%Y") 
activelimit=(current_date - timedelta(days=35)).strftime("%m-%d-%Y")
al=datetime.strptime(activelimit, "%m-%d-%Y")
for i in range(len(jsondata)):    
    userid=jsondata[i]['UserId']['S']
    firstname=jsondata[i]['FirstName']['S']
    lastname=jsondata[i]['LastName']['S']
    email=jsondata[i]['Email']['S']
    active=None
    try:
       last_sesh=jsondata[i]['Sessions']['M']['Core']['L'][-1]['M']['Date']['S']       
       last_sesh=pd.to_datetime(last_sesh)
       sesh_count=len(jsondata[i]['Sessions']['M']['Core']['L'])
       if last_sesh > al:
           active=True        
    except:
        last_sesh=None        
        sesh_count=0
    finally:
        metrics.append([userid,firstname,lastname,email,last_sesh,sesh_count,active])
columns=['UserId','FirstName','LastName','Email','LastSession','SessionCount','Active']
metrics_df=pd.DataFrame(metrics,columns=columns)
metrics_df=metrics_df.dropna().reset_index(drop=True)
alert=[]
list = [4,9,24,49,74,99,124,149,199]
for k, v in metrics_df['SessionCount'].items():
    if v in list:
        alert.append(metrics_df.loc[k,['FirstName','LastName','SessionCount']])
alert=pd.DataFrame(alert,columns=['FirstName','LastName','SessionCount']).sort_values(by='SessionCount',ascending=False)
alert.reset_index(drop=True,inplace=True)
alert.to_excel(f'/Users/dreamscape08/Desktop/OS_METRICS/Osteostrong Project/OS_Operations_Reports/LW-{current_date}.xlsx', header=True,index=False)
username = config('pg_username')
password = config('pg_password')
host = 'localhost'
port = '5432'
database = 'osteostrong'
postgres_str = f'postgresql://{username}:{password}@{host}:{port}/{database}'
cnx=create_engine(postgres_str)
metrics_df.to_sql('lw_Clientdb', cnx, if_exists='replace',index=False)
operations_manager = config('opm_email')
sender_proton = config('proton_email')
p_key = config('proton_key')
to = f'{operations_manager}'
admin_email = f'{sender_proton}' # (You should provide your gmail account name)
admin_pass = f'{p_key}' # (You should provide your gmail password)

pathnow=f'/Users/dreamscape08/Desktop/OS_METRICS/Osteostrong Project/OS_Operations_Reports/LW-{current_date}.xlsx'
def send_mail_with_excel():
    msg = EmailMessage()
    msg['Subject'] = f'LW FUB ALERTS-{current_date}'
    msg['From'] = admin_email
    msg['To'] = to
    msg.set_content(f'{alert}')

    with open(pathnow, 'rb') as f:
        file_data = f.read()
    msg.add_attachment(file_data, maintype="application", subtype="vnd.openxmlformats-officedocument.spreadsheetml.sheet", filename=f'LW_FUB_{current_date}.xlsx')

    with smtplib.SMTP('localhost',1025) as smtpserver:
        smtpserver.ehlo()
        smtpserver.starttls()
        smtpserver.ehlo
        smtpserver.login(admin_email, admin_pass)
        smtpserver.send_message(msg)
        print('Email sent!')
        smtpserver.close() 
send_mail_with_excel()

Email sent!
